In [2]:
import numpy as np
import os
import pandas as pd
import scipy.cluster.hierarchy as shc
import matplotlib.pyplot as plt
import random
import seaborn as sns
%matplotlib inline
from scipy.cluster.hierarchy import fcluster
import h5py
from io import StringIO
from scipy.cluster.hierarchy import cophenet
from scipy.spatial.distance import pdist
import itertools
import xlsxwriter as Excel

In [28]:
data_path = "../data/raw"
#converts to df to binary dataframe, 0 representing that the previous value was 0, and 1 indicating a nonzero value
def df_to_binary(df_expression):    
    temp = df_expression.copy()
    num_rows = len(temp)
    for c in range(len(df_expression.columns)):
        nonzero_inds = df_expression.iloc[:,c].to_numpy().nonzero()[0]
        temp_col = np.zeros(num_rows)
        temp_col[nonzero_inds] = 1
        temp.iloc[:,c] = temp_col
    return(temp)



def normalize(X):
    return (X - np.mean(X,axis=1).reshape(X.shape[0],1))/np.linalg.norm((X - np.mean(X,axis=1).reshape(X.shape[0],1)),ord=2,axis=1).reshape(X.shape[0],1)


#INPUTS:
#  file_list: list of csv filenames containing expression data. Will concatenate these matrices together and compute distances
def preprocess_samples(file_list):
    sample_source = []
    # Merge all of the files into one dataframe
    df_expression = pd.read_csv(os.path.join(data_path,file_list[0]),index_col = 0,sep="\t")
    sample_source.extend(np.repeat(file_list[0],len(df_expression.columns)))
    for filename in file_list[1:]:
        temp = pd.read_csv(os.path.join(data_path,filename),index_col = 0, sep = "\t")
        sample_source.extend(np.repeat(filename,len(temp.columns)))
        df_expression = df_expression.merge(temp,right_index=True,left_index=True)
    
    df_expression.columns = pd.Series(df_expression.columns)
    df_expression.index = pd.Series(df_expression.index)
    
    return(df_expression,sample_source)    

def hierarchical_clustering(df,title,method="average",metric="euclidean"):
    clustering = shc.linkage(df, method=method, metric=metric)
    plt.figure(figsize=(25, 20))  
    plt.title(title)  

    dend = shc.dendrogram(clustering) 
    print("Cophenet = " + str(cophenet(clustering, pdist(df))[0]))
    return(clustering)

#INPUTS: 
#  h5filepath: path where the full matrix is stored with meta and expression data
#  subsamples: a pandas DataFrame that must contain a column (foreign key) called SampleGeoAccession
#OUTPUT
#  merged dataframe of length(subsamples) merged with relevant meta/Sample values for cluster analysis
def h5_sample_meta_lookup(h5filepath,subsamples):
    f = h5py.File(h5filepath, 'r')
    full_data = pd.DataFrame({"Description":f[("meta/Sample_description")], 
                          "Characteristics":f[("meta/Sample_characteristics_ch1")],
                          "SampleGeoAccession":f[("meta/Sample_geo_accession")],
                          "Series ID":f[("meta/Sample_series_id")],
                          "Molecule":f["meta/Sample_molecule_ch1"],
                          "Source Name":f["meta/Sample_title"]})
    f.close()
    full_data["SampleGeoAccession"] = full_data["SampleGeoAccession"].apply(lambda x : x.decode("utf-8"))
    cluster_meta = subsamples.merge(full_data,on="SampleGeoAccession",how="left")
    cluster_meta["Sample_Label"] = pd.Series(sample_source).str.replace(".csv","").str.replace("_expression"," ")
    cluster_meta["Series ID"] = cluster_meta["Series ID"].str.decode("utf-8").str.replace("Xx","").str.replace("xX","")
    cluster_meta["Molecule"] = cluster_meta["Molecule"].str.decode("utf-8")
    cluster_meta["Source Name"] = cluster_meta["Source Name"].str.decode("utf-8")
    cluster_meta["Description"] = cluster_meta["Description"].str.decode("utf-8").str.replace("Xx","").str.replace("xX","")
    cluster_meta["Characteristics"] = cluster_meta["Characteristics"].str.decode("utf-8").str.replace("Xx","").str.replace("xX","")
    return(cluster_meta)

def purity_to_excel(cluster_metadata_metrics, H0, outfile):
    workbook = Excel.Workbook("tables/" +outfile +".xlsx")
    worksheet = workbook.add_worksheet()
    cluster_format = workbook.add_format({'bold': True, 'align':'center'})
    center = workbook.add_format({'align': 'center'})

    # Add a format. Light red fill with dark red text.
    format1 = workbook.add_format({'bg_color': '#FFC7CE',
                                   'font_color': '#9C0006'})

    # Add a format. Green fill with dark green text.
    format2 = workbook.add_format({'bg_color': '#C6EFCE',
                                   'font_color': '#006100'})

    worksheet.write('A1', "Cluster ID",cluster_format)
    worksheet.write('B1', "Cluster Size",center)
    worksheet.write('C1', "Dominant Molecule",center)
    worksheet.write('D1', "Molecule Purity",center)
    worksheet.write('E1', "Dominant Series",center)
    worksheet.write('F1', "Series Purity",center)
    worksheet.write('G1', "Dominant Label",center)
    worksheet.write('H1', "Label Purity",center)
    worksheet.write('I1', "Null Hypothesis Value",center)

    worksheet.set_column(1, 7, 15)

    num_keys = len(list(cluster_metadata_metrics.keys()))

    j = 1
    for i in np.sort(list(cluster_metadata_metrics.keys())):
        worksheet.write(j,0,i,center)
        worksheet.write(j,1,cluster_metadata_metrics[i]["Size"],center)
        worksheet.write(j,2,list(cluster_metadata_metrics[i]["Molecule Purity"].keys())[0],center)
        worksheet.write(j,3,list(cluster_metadata_metrics[i]["Molecule Purity"].values())[0],center)
        worksheet.write(j,4,list(cluster_metadata_metrics[i]["Series Purity"].keys())[0],center)
        worksheet.write(j,5,list(cluster_metadata_metrics[i]["Series Purity"].values())[0],center)
        worksheet.write(j,6,list(cluster_metadata_metrics[i]["Label Purity"].keys())[0],center)
        worksheet.write(j,7,list(cluster_metadata_metrics[i]["Label Purity"].values())[0],center)
        worksheet.write(j,8,H0[list(cluster_metadata_metrics[i]["Label Purity"].keys())[0]],center)
        
        

        # Write a conditional format over a range.
        worksheet.conditional_format('H'+str(j+1), {'type': 'cell',
                                             'criteria': '>=',
                                             'value': H0[list(cluster_metadata_metrics[i]["Label Purity"].keys())[0]],
                                             'format': format2})

        # Write another conditional format over the same range.
        worksheet.conditional_format('H'+str(j+1), {'type': 'cell',
                                             'criteria': '<',
                                             'value': H0[list(cluster_metadata_metrics[i]["Label Purity"].keys())[0]],
                                             'format': format1})

        j += 1

    workbook.close()


def get_purity(cluster_assignments, df_expression_trans):
    cluster_dict = {"SampleGeoAccession":[],"ClusterAssignment":[]}
    cluster_dict["SampleGeoAccession"] = df_expression_trans.index
    cluster_dict["ClusterAssignment"] = cluster_assignments
    cluster_info = pd.DataFrame(cluster_dict)
    cluster_metadata = h5_sample_meta_lookup("human_matrix.h5",cluster_info).sort_values("ClusterAssignment")
    
    #hypothesized values of expected sample frequencies assuming no separation (based on observed proportion of sample labels)
    H0 = {}
    for samp in cluster_metadata.Sample_Label:
        H0[samp] = len(cluster_metadata[cluster_metadata.Sample_Label == samp])/len(cluster_metadata)
    
    cluster_metadata_metrics = {}
    for c in range(1,k+1):
        cluster_metadata_metrics[c] = {"Size":len(cluster_metadata.loc[cluster_metadata.ClusterAssignment == c]), 
                                       "Molecule Purity":{},"Series Purity":{},"Label Purity":{}}
        series_counts = cluster_metadata.groupby('ClusterAssignment')["Series ID"].value_counts()
        molecule_counts = cluster_metadata.groupby('ClusterAssignment')["Molecule"].value_counts()
        label_counts = cluster_metadata.groupby('ClusterAssignment')["Sample_Label"].value_counts()

        series_purity_key = pd.Series(series_counts[c][series_counts[c] == series_counts[c].max()]).index[0]
        series_purity_val = series_counts[c].max()/float(series_counts[c].sum())
        cluster_metadata_metrics[c]["Series Purity"][series_purity_key] = round(series_purity_val,3)

        molecule_purity_key = molecule_counts[c][molecule_counts[c] == molecule_counts[c].max()].index[0]
        molecule_purity_val = molecule_counts[c].max()/float(molecule_counts[c].sum())
        cluster_metadata_metrics[c]["Molecule Purity"][molecule_purity_key] = round(molecule_purity_val,3)

        label_purity_key = label_counts[c][label_counts[c] == label_counts[c].max()].index[0]
        label_purity_val = label_counts[c].max()/float(label_counts[c].sum())
        cluster_metadata_metrics[c]["Label Purity"][label_purity_key] = round(label_purity_val,3)
    purity_to_excel(cluster_metadata_metrics,H0,cluster_title+" purity")

In [ ]:
file_list = ["HT1080_expression.csv", "HCT116_expression.csv","Colon_expression.csv"]
df_expression_raw,sample_source = preprocess_samples(file_list)

gene_list = df_expression_raw.index
sample_list  =df_expression_raw.columns

filtering

In [ ]:
random_inds = list(np.random.choice(range(37,1572),200)) + list(range(37))
df_expression_raw = df_expression_raw.iloc[random_inds,:]
sample_source = np.array(sample_source)[random_inds]
gene_list = df_expression_raw.index
sample_list  =df_expression_raw.columns

In [ ]:
gene_list = ['AC004824.1', 'AC006427.2', 'AC010740.1', 'AC093084.1',
       'AC130360.7', 'ADAM30', 'AF196972.4', 'AK9', 'AL355480.2', 'AP2M1',
       'ARPC1A', 'ATG16L2', 'ATP5F1', 'BCL2L15', 'BCRP1', 'C12ORF66',
       'CDR1', 'CEP128', 'CGA', 'CH17-13I23.3', 'CH507-236L23.4',
       'CNN2P10', 'CTB-179I1.1', 'CTC-503J8.2', 'CTD-2014N11.1',
       'CTD-2501O3.2', 'DIMT1', 'DLST', 'DMXL1', 'DNAJB1', 'EARS2',
       'EIF4A1P9', 'FXYD1', 'GOLGA6L4', 'GPATCH8', 'GZMAP1', 'HEMK1',
       'HMGB1P29', 'HS3ST6', 'IGHV2OR16-5', 'ITFG1', 'IYD', 'KDM4E',
       'MCRS1', 'MT-CYB', 'NOL7', 'NUTF2P4', 'OR10A3', 'OR2L13',
       'OR51A9P', 'OR9K2', 'PROSER2', 'PRSS53', 'REG4', 'RNPEP',
       'RP1-20N2.7', 'RP11-404F10.6', 'RP11-411B10.4', 'RP11-548K12.6',
       'RP11-989E6.13', 'RP3-509I19.1', 'RP3-511B24.5', 'RP4-593A12.1',
       'RPL23AP48', 'RPL23AP54', 'RPL34P27', 'RPL36AL', 'SEC62', 'SETD9',
       'SH3GL1P3', 'SLC10A6', 'SMC1B', 'STX7', 'SUGP1', 'TBX1', 'TGIF2',
       'TPPP2', 'TUBB8P2', 'TUBBP10', 'USP17L28', 'VKORC1L1', 'WFDC6',
       'WHRN', 'XCL2', 'ZC3H18', 'ZNF354C', 'ZNF449', 'ZNF705G']

sample_list = df_expression_raw.columns

df_expression_raw = df_expression_raw.loc[gene_list,sample_list]

In [ ]:
gene_list = ['AC004824.1', 'AC006427.2', 'AC010740.1', 'AC093084.1',
       'AC130360.7', 'ADAM30', 'AF196972.4', 'AK9', 'AL355480.2', 'AP2M1',
       'ARPC1A', 'ATG16L2', 'ATP5F1', 'BCL2L15', 'BCRP1', 'C12ORF66',
       'CDR1', 'CEP128', 'CGA', 'CH17-13I23.3', 'CH507-236L23.4',
       'CNN2P10', 'CTB-179I1.1', 'CTC-503J8.2', 'CTD-2014N11.1',
       'CTD-2501O3.2', 'DIMT1', 'DLST', 'DMXL1', 'DNAJB1', 'EARS2',
       'EIF4A1P9', 'FXYD1', 'GOLGA6L4', 'GPATCH8', 'GZMAP1', 'HEMK1',
       'HMGB1P29', 'HS3ST6', 'IGHV2OR16-5', 'ITFG1', 'IYD', 'KDM4E',
       'MCRS1', 'MT-CYB', 'NOL7', 'NUTF2P4', 'OR10A3', 'OR2L13',
       'OR51A9P', 'OR9K2', 'PROSER2', 'PRSS53', 'REG4', 'RNPEP',
       'RP1-20N2.7', 'RP11-404F10.6', 'RP11-411B10.4', 'RP11-548K12.6',
       'RP11-989E6.13', 'RP3-509I19.1', 'RP3-511B24.5', 'RP4-593A12.1',
       'RPL23AP48', 'RPL23AP54', 'RPL34P27', 'RPL36AL', 'SEC62', 'SETD9',
       'SH3GL1P3', 'SLC10A6', 'SMC1B', 'STX7', 'SUGP1', 'TBX1', 'TGIF2',
       'TPPP2', 'TUBB8P2', 'TUBBP10', 'USP17L28', 'VKORC1L1', 'WFDC6',
       'WHRN', 'XCL2', 'ZC3H18', 'ZNF354C', 'ZNF449', 'ZNF705G']
sample_list = ['GSM1093236', 'GSM1649207', 'GSM1649201', 'GSM1649199',
       'GSM1649206', 'GSM1649198', 'GSM1649205', 'GSM1649204',
       'GSM1649208', 'GSM1649203', 'GSM1649197', 'GSM1649195',
       'GSM1649191', 'GSM1649196', 'GSM1649212', 'GSM1980055',
       'GSM1980058', 'GSM1980057', 'GSM1980056', 'GSM1649194',
       'GSM3430684', 'GSM3430685', 'GSM2279678', 'GSM2279679',
       'GSM2279680', 'GSM2279681', 'GSM2279682', 'GSM2279683',
       'GSM2279684', 'GSM2279685']

sample_source = ["ALPHA","ALPHA","ALPHA","ALPHA","ALPHA","ALPHA","ALPHA","ALPHA","ALPHA","ALPHA","beta",
                "beta","beta","beta","beta","beta","beta","beta","beta","beta","PANC1",
                "PANC1","PANC1","PANC1","PANC1","PANC1","PANC1","PANC1","PANC1","PANC1",]
df_expression_raw = df_expression_raw.loc[gene_list,sample_list]

In [ ]:
nonzero_inds = (df_expression_raw.T != 0).any()
df_nonzero = df_expression_raw[nonzero_inds]
sample_source = np.array(sample_source)[nonzero_inds]
gene_list = gene_list[nonzero_inds]
df_expression_norm = pd.DataFrame(np.transpose(normalize(np.transpose(np.array(df_nonzero)))),columns=sample_list,index=gene_list)


In [ ]:
method="average"
metric="euclidean"
#df_expression_trans = np.transpose(df_expression_norm)
df_expression_trans = np.transpose(df_expression_raw)
#df_expression_trans = np.transpose(df_to_binary(df_expression_trans))
dataTitle="Normalized"
cluster_title = dataTitle +" Data (Filtered) Ovary vs. SKOV3 cell "+method+"+"+metric
df_expression_trans = df_expression_norm
genes = list(df_expression_trans.columns)
samples = list(df_expression_trans.index)

In [ ]:
clustering = hierarchical_clustering(df_expression_trans, method=method, metric=metric, title = cluster_title + " Dendrogram")

determine k from above dendrogram

In [ ]:
k = 4
cluster_assignments = fcluster(clustering, k, criterion='maxclust')

outputs an excel file

In [ ]:
get_purity(cluster_assignments, df_expression_trans)

# The full pipeline in one function call

In [27]:
#Inputs

h5_filepath = "../data/raw/human_matrix.h5"

samples_filter_to = ["GSM742947","GSM979870","GSM1196045","GSM979869","GSM1505605","GSM1548004","GSM1548001","GSM1548003","GSM1505580","GSM1547996","GSM1548002","GSM1548000","GSM1547998","GSM1547997","GSM1547999","GSM2343138","GSM1101662","GSM2344226","GSM1010948","GSM2453417","GSM2343109","GSM3415785","GSM3415786","GSM3415852","GSM3415878","GSM3613418","GSM3613419","GSM3613420","GSM3617822","GSM3319032","GSM3319033",
"GSM3319034","GSM3319035","GSM3319036","GSM3319037","GSM3319038","GSM3319039","GSM3319040","GSM3319041","GSM3319042","GSM3319043","GSM3319044","GSM3319045","GSM3319046","GSM3319047","GSM3557967","GSM3557969","GSM3557973","GSM3944441","GSM3944442","GSM3944443","GSM3944444","GSM3944445","GSM3944446","GSM3944447","GSM3944448","GSM3944449","GSM3944450","GSM3944451","GSM3944452","GSM3944453",
"GSM3944454","GSM3944455","GSM3944456","GSM3944457","GSM3944458","GSM3944459","GSM3944460","GSM3944461","GSM3944462","GSM3944463","GSM3944464","GSM3089935","GSM3089936","GSM3089937","GSM3089938","GSM3089946","GSM4073729","GSM4073731","GSM4073733","GSM4073735","GSM4073737","GSM4073739","GSM4073741","GSM4073743","GSM4073745","GSM4073747","GSM4073749","GSM4073752","GSM4073754","GSM4073756",
"GSM4073758","GSM4073760","GSM4073762","GSM4073764","GSM4073766","GSM4110159","GSM4110160","GSM4110161"]

sample_source = np.repeat("ovary", len(samples_filter_to))

sample_labels = np.repeat("Ovary",len(samples_filter_to))

genes_filter_to = None


#Functional Code

#obtain submatrix based on provided filter_to lists
with h5py.File(h5_filepath, "r") as h5file:
    h5_matrix = h5file['data']['expression']
    
    num_samples = h5_matrix.shape[0]
    num_genes = h5_matrix.shape[1]
    
    all_samples = np.array(h5file["meta/Sample_geo_accession"])
    all_genes = np.array(h5file["meta/genes"])

    
    if samples_filter_to is None:
        print("No sample filter is specified, do you wish to proceed to cluster on all %d samples?" % num_samples)

        if genes_filter_to is None:
            print("No gene filter is specified, do you wish to proceed to cluster on all %d genes?" % num_genes)    
            h5_submatrix = h5_matrix
        else:
            h5_submatrix = h5_matrix[:,np.isin(all_genes,genes_filter_to)]
        
        
        if sample_labels is None:
            sample_labels = h5file["meta/Sample_source_name_ch1"]


    else:
        if genes_filter_to is None:
            print("No gene filter is specified, do you wish to proceed to cluster on all %d genes?" % num_genes)    
            h5_submatrix = h5_matrix[np.isin(all_samples,samples_filter_to),:]
        else:
            h5_submatrix = h5_matrix[np.isin(all_samples,samples_filter_to),all_samples,np.isin(all_genes,genes_filter_to)]

            
        if sample_labels is None:
            sample_labels = h5file["meta/Sample_source_name_ch1"][np.isin(all_samples,samples_filter_to)]
    
    
print(h5_submatrix.shape)
print(len(sample_labels))
    
    
    

No gene filter is specified, do you wish to proceed to cluster on all 35238 genes?
(99, 35238)
99


In [29]:
n = normalize(h5_submatrix)

In [13]:

def gene_expression_cluster_analysis(h5_filepath,
                                     samples_filter_to = None,
                                     genes_filter_to = None,
                                     sample_labels = None,
                                     value_transformation = "raw",
                                     hier_method = "average",
                                     hier_distance_metric = "euclidean",
                                     hier_k = 4,
                                     hier_fcluster_t = 4):
    
    """
Performs gene expression cluster analysis using one or combination of hierarchical, k-means, and SPC.


Arguments:

    h5_filepath (str): filepath to h5 file of interest (human, mouse)
    
    samples_filter_to (list [str]): list of samples to filter to in matrix. 
                       If None, then all samples will be analyzed
                       
    genes_filter_to (list [str]): list of genes to filter to in matrix.
                       If None, then all genes will be analyzed
    
    sample_labels (list [str]): list of same length as samples_filter_to that 
                   labels the source tissue/cell line from which
                   the sample was obtained from. If None, then
                   Sample_source_name_ch1 from h5 metadata will
                   be used
                   
    value_transformation (str): how to transform the values in the h5 matrix
                          can be one of ["raw","normalize","binary"]
                          "raw" by default
                          TODO: implement library size,
                                          DESeq, 
                                          Relative Log Expression (RLE), 
                                          upper quartile (UQ), 
                                          Trimmed Mean of M values (TMM),
                                          Total Ubiquitous (TU), 
                                          Network Centrality Scaling (NCS), 
                                          Evolution Strategy (ES),
                                          pooled size factors (Pooled) (Lun et. al),
                                          SCnorm
    hier_method (str): specify the method of hierarhcical linkage to be one of 
                 ('average', 'complete', 'single', 'weighted', 'centroid', 'median' , 'ward')
    
    hier_distance_metric (str): specify the distance metric used in hierarchical clustering to be one of:
                                (‘braycurtis’, ‘canberra’, ‘chebyshev’, ‘cityblock’, ‘correlation’, ‘cosine’, 
                                ‘dice’, ‘euclidean’, ‘hamming’, ‘jaccard’, ‘jensenshannon’, ‘kulsinski’, ‘mahalanobis’, 
                                ‘matching’, ‘minkowski’, ‘rogerstanimoto’, ‘russellrao’, ‘seuclidean’, ‘sokalmichener’, 
                                ‘sokalsneath’, ‘sqeuclidean’, ‘yule’)
                 
    
    hier_k (int): Number of clusters to output from hierarchical clustering. Default to 4
    
    hier_fcluster_metric (str): one of ('inconsistent','distance','maxclust','monocrit','maxclust_monocrit')
    
    hier_fcluster_t (float): https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.cluster.hierarchy.fcluster.html
                                    
    Returns:

    hier_cluster_dendrogram (matplotlib.pyplot): dendrogram of chosen hierarchical clustering procedure
    """
    
    #get submatrix based on filter_to arguments
    h5_submatrix, sample_labels = filter_matrix(h5_filepath,samples_filter_to,genes_filter_to)
    #make sure sample labels is same length as number of samples in submatrix
    #otherwise a data entry error occurred and we need to terminate
    if h5_submatrix.shape[0] != len(sample_labels):
        print("Error: provided sample_labels not the correct length. Terminating program")
        return
    
    #Go through possible value normalizing transformations
    
    if value_transformation == "normalize":
        transformed_submatrix = normalize(h5_submatrix)
    #other normalizations to implement
    #elif value_transformation == "":
    else:
        transformed_submatrix = h5_submatrix
        
        
    hier_title=hier_method+" Link Hierarchical Dendrogram, "+hier_distance_metric+", "+value_transformation    
    hier_clustering = hierarchical_clustering(transformed_submatrix, 
                                              method=hier_method, 
                                              metric=hier_metric, 
                                              title=hier_title)
        
    hier_cluster_assignments = fcluster(clustering, hier_k, criterion='maxclust')
    

"""
Performs Hierarchical Clustering and returns

"""
def hierarchical_clustering(df,title,method="average",metric="euclidean"):
    clustering = shc.linkage(df, method=method, metric=metric)
    #plt.figure(figsize=(25, 20))  
    #plt.title(title)  

    #dend = shc.dendrogram(clustering) 
    print("Cophenet = " + str(cophenet(clustering, pdist(df))[0]))
    return(clustering)

"""
normalizes (L2) a gene expression matrix such that sum of values for each sample equals 0
and each row (sample) has same standard deviation
"""
def normalize(X):
    return (X - np.mean(X,axis=1).reshape(X.shape[0],1))/np.linalg.norm((X - np.mean(X,axis=1).reshape(X.shape[0],1)),ord=2,axis=1).reshape(X.shape[0],1)

#converts to df to binary dataframe, 0 representing that the previous value was 0, and 1 indicating a nonzero value
def df_to_binary(df_expression):    
    temp = df_expression.copy()
    num_rows = len(temp)
    for c in range(len(df_expression.columns)):
        nonzero_inds = df_expression.iloc[:,c].to_numpy().nonzero()[0]
        temp_col = np.zeros(num_rows)
        temp_col[nonzero_inds] = 1
        temp.iloc[:,c] = temp_col
    return(temp)


def filter_matrix(h5_filepath,
                  samples_filter_to, 
                  genes_filter_to, 
                  sample_labels):
    
    #obtain submatrix based on provided filter_to lists
    with h5py.File(h5_filepath, "r") as h5file:
        h5_matrix = h5file['data']['expression']

        num_samples = h5_matrix.shape[0]
        num_genes = h5_matrix.shape[1]

        all_samples = np.array(h5file["meta/Sample_geo_accession"])
        all_genes = np.array(h5file["meta/genes"])


        if samples_filter_to is None:
            print("No sample filter is specified, do you wish to proceed to cluster on all %d samples?" % num_samples)

            if genes_filter_to is None:
                print("No gene filter is specified, do you wish to proceed to cluster on all %d genes?" % num_genes)    
                h5_submatrix = h5_matrix
            else:
                h5_submatrix = h5_matrix[:,np.isin(all_genes,genes_filter_to)]


            if sample_labels is None:
                sample_labels = h5file["meta/Sample_source_name_ch1"]


        else:
            if genes_filter_to is None:
                print("No gene filter is specified, do you wish to proceed to cluster on all %d genes?" % num_genes)    
                h5_submatrix = h5_matrix[np.isin(all_samples,samples_filter_to),:]
            else:
                h5_submatrix = h5_matrix[np.isin(all_samples,samples_filter_to),all_samples,np.isin(all_genes,genes_filter_to)]


            if sample_labels is None:
                sample_labels = h5file["meta/Sample_source_name_ch1"][np.isin(all_samples,samples_filter_to)]

    return(h5_submatrix,sample_labels)

19044021

# Gene Shaving

In [ ]:
file_list = ["ALPHA_expression.csv","beta_expression.csv","PANC1_expression.csv"]
df,sample_source = preprocess_samples(file_list)

In [ ]:
#Let X be N (genes) x P (samples) expression matrix (pandas dataframe)
def gene_shave(df):
    #clusters where clusterID (key) also represents the size k, values are a list of inds of genes belonging to cluster
    S = {}
    xbarSk = {}
    #remove rows of all zeros
    df_nonzero = df[(df.T != 0).any()]
    genes = np.array(df_nonzero.index)
    samples = list(df_nonzero.columns)
    X = np.array(df_nonzero)
    X = (X - np.mean(X,axis=1).reshape(X.shape[0],1))/np.linalg.norm((X - np.mean(X,axis=1).reshape(X.shape[0],1)),ord=2,axis=1).reshape(X.shape[0],1)
    X_copy = X.copy()
    while(X_copy.shape[0] > 1):
        corr = np.corrcoef(np.transpose(X_copy))
        w,v = np.linalg.eig(corr)
        #leading principal component is vmax
        vmax = v[0]
        dot_prods = np.dot(X_copy,vmax)
        dot_prods_argsort = np.argsort(dot_prods)
        #save the inds of top 90% of rows with respect to dot product with vmax
        top90perc = np.where(dot_prods_argsort > len(dot_prods_argsort)*0.1)[0]
        S[len(top90perc)] = genes[top90perc]
        xbarSkhat = np.mean(X_copy[top90perc,:],axis=0)
        xbarSk[len(top90perc)] = xbarSkhat
        #remove the bottom 10% of X_copy's rows with respect to dot product with vmax
        X_copy = X_copy[top90perc,:]
        genes = genes[top90perc]
    return(S,xbarSk)

In [ ]:
S, xbarSk = gene_shave(df)

In [ ]:
def Dk(df,k,xbar):
    Dks = []
    sumVw = 0.0
    sumVb = 0.0
    #sum over p columns
    jcnt = 0
       
    for j in df.columns: 
        #sum over each gene in Sk
        #print("OK" + str(xbar.shape))
        sumVw = np.sum((df.loc[:,j] - xbar[jcnt])**2)
        #or i in S[k]:
        #    sumVw += (df.loc[i,j] - xbar[jcnt])**2
        sumVb = np.sum((xbar[jcnt] - 0)**2)
        jcnt += 1
    Vw = sumVw/(k*len(df.columns))
    Vb = sumVb/len(df.columns)
    Vt = Vw+Vb
    Dks.append((Vb/Vw)/(1+(Vb/Vw)))
    return(Dks[0])

def findK(S, xbarSk, df, B=5):
    
    Gapk = {}
    Dks = {}
    D_k_b = {}
    for k in S:
        print(k)
        D_k_b[k] = []
        D_k = Dk(df.loc[S[k],:],k,xbarSk[k])
        for i in range(B):
            df_copy = df.sample(k,replace=False)
            xbar = np.mean(np.array(df_copy),axis=0)
            D_k_b[k].append(Dk(df,k,xbar))
        Dks[k] = (D_k)
    return(Dks,D_k_b)
        
                
                

In [ ]:
#find the optimum value of k based on R^2 values. Choose the value of k with the highest value
df_nonzero = df[(df.T != 0).any()]
genes = np.array(df_nonzero.index)
X = np.array(df_nonzero)
X = (X - np.mean(X,axis=1).reshape(X.shape[0],1))/np.linalg.norm((X - np.mean(X,axis=1).reshape(X.shape[0],1)),ord=2,axis=1).reshape(X.shape[0],1)
df_norm = pd.DataFrame(X,index=df_nonzero.index,columns=df_nonzero.columns)
Dks,Dkbs = findK(S, xbarSk , df_norm)

Select genes for visualization purposes (k=6 seems to be the winner)

In [ ]:
gapK = {}
for i in Dks:
    gapK[i] = Dks[i] - np.mean(Dkbs[i])

In [ ]:
gapK

In [ ]:
S[80]

In [ ]:
list(df_nonzero.columns)

# Transfer fcluster output to csv for R heatmap

In [ ]:
def sortDF(df, cluster_assignments, outfile):
    sample_list_sorted = []
    for i in np.unique(cluster_assignments):
        for j in list(df.index[cluster_assignments == i]):
            sample_list_sorted.append(j)
    df.loc[sample_list_sorted,:].to_csv(outfile)  

In [ ]:
sortDF(np.transpose(df_norm),fcluster(clustering, len(sample_list), criterion='maxclust'),"haha.csv")

In [ ]:
fcluster(clustering, len(df_expression_trans), criterion='maxclust')

In [ ]:
np.unique(cluster_assignments)

In [ ]:
df_expression_trans

In [ ]:
np.unique(test_ass)

In [ ]:
sample_list_sorted = []
for i in np.unique(test_ass):
    for j in test_ass[test_ass == i]:
        sample_list_sorted.append(sample_list[i])
df_expression_trans.loc[sample_list_sorted,:]    

In [ ]:
sample_list_sorted

# XGBoost classification

In [ ]:
from xgboost import XGBClassifier
import sqlite3 #sqlite will suffice due to small magnitude of projected data
from sqlite3 import Error
from sklearn import linear_model
from sklearn.svm import SVC
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

In [ ]:
file_list = ["PANC1_expression.csv","ALPHA_expression.csv","beta_expression.csv"]
df,sample_source = preprocess_samples(file_list)

In [ ]:
genelist = ['AC114755.7', 'APBA1', 'ARNTL', 'C11ORF98', 'C20ORF85', 'C21ORF62',
       'C2ORF54', 'CCDC166', 'CDK3', 'CERS5', 'CTC-459F4.7', 'DDR1',
       'DGKK', 'ELF2', 'FAM206BP', 'FAM90A13P', 'FDCSP', 'FGF7P1',
       'FLYWCH2', 'FOXK2', 'GABBR2', 'GADD45B', 'GIGYF1', 'GIMAP3P',
       'GJA4', 'GKAP1', 'GOLGA8J', 'GRM2', 'GSTA7P', 'GUSBP6', 'HMG20B',
       'HMGA1P2', 'HNRNPA3P15', 'HNRNPH1P2', 'HSFY4P', 'IGHV3-13',
       'IGKV2D-14', 'IGKV3-31', 'KCNA6', 'LCE2B', 'LPXN', 'MCM2', 'MED27',
       'MICU2', 'MTCO3P35', 'NECAP1', 'NPY', 'NTM', 'PCDHGA1', 'PHLDB3',
       'PHLPP2', 'PIWIL3', 'PPIL1', 'PRODH', 'RERGL', 'RIMBP3B', 'RNF20',
       'RP11-102M11.1', 'RP11-117N2.2', 'RP11-163O19.8', 'RP11-170N16.2',
       'RP11-252A24.2', 'RP11-345J4.5', 'RP11-463J7.3', 'RP11-56P9.8',
       'RP11-69M1.3', 'RP11-72M10.8', 'RP11-790I12.3', 'RP11-848P1.9',
       'RP6-149D17.1', 'RPL12P39', 'RPL17-C18ORF32', 'RPL22P11',
       'RPL31P9', 'SLC27A2', 'SUMO2P7', 'TEX37', 'TSPEAR', 'UBL4B',
       'ZNF271P']

df = df.loc[genelist,]

In [ ]:
def compare_models_cv(model_dict, X_dict, y, cv = 4, scoring = ["accuracy", "f1_micro", "f1_macro"]):
    output_dict = {}
    for model_name in list(model_dict.keys()):
        for X_name in list(X_dict.keys()):
            model = model_dict[model_name]
            print(model)
            X = X_dict[X_name]
            #print(X)
            output_dict[model_name + " + " + X_name] = {"accuracy":None, "f1_micro":None, "f1_macro":None}
            for score in scoring:
                scores = cross_val_score(model, X, y, cv=6, scoring = score)
                output_dict[model_name + " + " + X_name][score] = scores.mean()
    return(output_dict)

In [ ]:
models = {"XGBoost eta0.1":XGBClassifier(eta = 0.1),
        "XGBoost eta0.2":XGBClassifier(eta = 0.2),
        "XGBoost eta0.3":XGBClassifier(eta = 0.3),
        "XGBoost md3":XGBClassifier(max_depth = 3),
        "XGBoost md4":XGBClassifier(max_depth = 4),
        "XGBoost md5":XGBClassifier(max_depth = 5),
        "XGBoost md6":XGBClassifier(max_depth = 6),
        "XGBoost md7":XGBClassifier(max_depth = 7)}

In [ ]:

inds = np.array(range(len(sample_source)))

inds = np.random.choice(inds,len(inds))

train_X = np.transpose(np.array(df.iloc[:,inds]))
train_y = np.array(pd.get_dummies(pd.Series(sample_source)).iloc[inds,0])

In [ ]:
train_y = []

y = np.array(pd.get_dummies(pd.Series(sample_source)))

for i in y:
    for k in range(3):
        if i[k]==1:
            train_y.append(k)

train_y = np.array(train_y)[inds]

In [ ]:
train_X = (train_X - np.mean(train_X,axis=0))/(np.max(train_X,axis=0) - np.min(train_X,axis=0))

In [ ]:
compare_models_cv(models,{"norm":train_X},train_y)

In [ ]:
pd.value_counts(sample_source)

In [ ]:
train_X.shape

In [ ]:
train_X[1,]

In [ ]:
train_X[615,:]

In [ ]:
train_X.shape

In [ ]:
sample_source[1]